In [1]:
import torch
import numpy as np
from model import cp_2_k_mask, cp_2_k_onnx
from config import args
from dataset import Enigma_simulate_c_2_p, Enigma_simulate_cp_2_k_limited, Enigma_simulate_cp_2_k
from torch.utils.data import DataLoader
import math
from torchsummary import summary
from tqdm import tqdm

Copying the compiled weight to regular models

In [2]:
# Load trained model's weights and config
ckpt = torch.load('CP2K_RNN_ENC_ckpt.pt')
ckpt_args = ckpt['args']

# Initialize new model by configs
model = cp_2_k_mask(args=ckpt_args, out_channels=26)
model.to('cuda')
model.eval()

# Load weights, calculate the number of parameters
weights = []
num_param = 0
for k, v in ckpt['weights'].items():
    weights.append(v)
    num_param += np.prod(v.shape)
print(f"Parameters: {num_param}")

# Copying weights from compile model to new model
for idx, (k, v) in enumerate(model.state_dict().items()):
    # We have to copying like this, I am so confused
    v *= 0
    v += weights[idx].detach()

# Set the model to evaluate mode
model.eval()

Parameters: 27408974


cp_2_k_mask(
  (networks): ModuleList(
    (0): RNN_encoder(
      (enc): LSTM(512, 512, num_layers=2, dropout=0.2, bidirectional=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (linear1): Linear(in_features=1024, out_features=2048, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=2048, out_features=1024, bias=True)
          (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.2, inplace=False)
          (dropout2): Dropout(p=0.2, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
           

Running a mini batch of examples

In [3]:
dataset = Enigma_simulate_cp_2_k_limited(args=args)
dataloader = DataLoader(
        dataset=dataset,
        batch_size=512,
        collate_fn=dataset.collate_fn_padding,
        shuffle=True
    )

data_iter = iter(dataloader)
inputs, targets, masks = next(data_iter)
inputs, targets, masks = inputs.to('cuda'), targets.to('cuda'), masks.to('cuda')

outputs = model(inputs, masks)

print(f"Input shape: {inputs.shape} {'[seq, batch, feats]'}\n"
          f"Target shape: {targets.shape}  {'[rotor, seq, batch]'}\n"
          f"Mask shape: {masks.shape}  {'[batch, seq]'}\n"
          f"Output shape: {outputs.shape}  {'[rotor, seq, batch, feats]'}")

true_positive = 0
samples = 0

outputs_indices = torch.argmax(outputs, dim=-1) # -> [rotor, seq, batch]
for rotor in range(outputs_indices.shape[0]):
    mask = outputs_indices[rotor][~masks.T] == targets[rotor][~masks.T]
    true_positive += mask.sum()
    samples += math.prod(mask.shape)

print(f"Acc: {true_positive / samples}")

Input shape: torch.Size([30, 512, 52]) [seq, batch, feats]
Target shape: torch.Size([3, 30, 512])  [rotor, seq, batch]
Mask shape: torch.Size([512, 30])  [batch, seq]
Output shape: torch.Size([3, 30, 512, 26])  [rotor, seq, batch, feats]
Acc: 0.9987196326255798


In [4]:
# Make prediction based on user's inputs
from dataset import cipher_plain_text_2_tensor
inputs, masks = cipher_plain_text_2_tensor('VMPDTAJYTXDZNEFOSOTPJOYSMO', 'WEARETHECHAMPIONANDTHEBEST') # Predict on usr inputs
inputs, masks = inputs.to('cuda'), masks.to('cuda')


# Make prediction
with torch.no_grad():
    outputs = model(inputs, masks).argmax(dim=-1).squeeze(-1) # -> shape [rotor, seq]

# print the prediction at the first postion, which is the initial states
print(f"Prediction: {chr(ord('A') + outputs[0, 0])}"
      f"{chr(ord('A') + outputs[1, 0])}"
      f"{chr(ord('A') + outputs[2, 0])}")

Prediction: AES


In [5]:
# Transfer to onnx
import onnx, onnxruntime
onnx_model = cp_2_k_onnx(args=ckpt_args, out_channels=26)
onnx_model.to('cuda')
onnx_model.eval()

# Load weights, calculate the number of parameters
weights = []
num_param = 0
for k, v in ckpt['weights'].items():
    weights.append(v)
    num_param += np.prod(v.shape)
print(f"Parameters: {num_param}")

# Copying weights from compile model to new model
for idx, (k, v) in enumerate(onnx_model.state_dict().items()):
    # We have to copying like this, I am so confused
    v *= 0
    v += weights[idx].detach()

# Testing the pre-export model in batch size 1
dataloader = DataLoader(
        dataset=dataset,
        batch_size=1,
        collate_fn=dataset.collate_fn_padding,
        shuffle=True
    )

# Get samples for dataset
data_iter = iter(dataloader)
inputs, targets, masks = next(data_iter)
inputs, targets, masks = inputs.to('cuda'), targets.to('cuda'), masks.to('cuda')

outputs = onnx_model(inputs)

print(f"Input shape: {inputs.shape} {'[seq, batch, feats]'}\n"
      f"Target shape: {targets.shape}  {'[rotor, seq, batch]'}\n"
      f"Mask shape: {masks.shape}  {'[batch, seq]'}\n"
      f"Output shape: {outputs.shape}  {'[rotor, seq, batch, feats]'}")


true_positive = 0
samples = 0

outputs_indices = torch.argmax(outputs, dim=-1) # -> [rotor, seq, batch]
for rotor in range(outputs_indices.shape[0]):
    mask = outputs_indices[rotor][~masks.T] == targets[rotor][~masks.T]
    true_positive += mask.sum()
    samples += math.prod(mask.shape)

print(f"Acc: {true_positive / samples}")

# Export onnx model
torch.onnx.export(
    onnx_model,
    args=inputs,
    f="seq5-45-large.onnx",
    input_names=["input_1"],
    output_names=["output1"]
)

# Load and verify onnx model
ort_session = onnxruntime.InferenceSession("seq5-45-large.onnx")
inputs, _ = cipher_plain_text_2_tensor('VMPDTAJYTXDZNEFOSOTPJOYSMOEBNX', 'WEARETHECHAMPIONANDTHEBESTPLAY')
outputs = ort_session.run(
    None,
    {"input_1": np.array(inputs)},
)

outputs = outputs[0].argmax(axis=-1).reshape(outputs[0].shape[0], outputs[0].shape[1])
print(outputs.shape)
print(f"Prediction: {chr(ord('A') + outputs[0, 0])}"
      f"{chr(ord('A') + outputs[1, 0])}"
      f"{chr(ord('A') + outputs[2, 0])}")


Parameters: 27408974
Input shape: torch.Size([30, 1, 52]) [seq, batch, feats]
Target shape: torch.Size([3, 30, 1])  [rotor, seq, batch]
Mask shape: torch.Size([1, 30])  [batch, seq]
Output shape: torch.Size([3, 30, 1, 26])  [rotor, seq, batch, feats]
Acc: 1.0


D:\Anaconda\envs\pytorch\lib\site-packages\torch\onnx\symbolic_opset9.py:2188: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn("Exporting a model to ONNX with a batch_size other than 1, " +


(3, 30)
Prediction: AES


In [ ]:
# Testint accuracy in different length
testing_args = args
results = {}


for length in range(5, 51):
    testing_args['SEQ_LENGTH'] = [length, length]
    dataset = Enigma_simulate_cp_2_k_limited(args=testing_args, mode='test')
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=1024,
        collate_fn=dataset.collate_fn_padding,
        shuffle=False,
        drop_last=False
    )

    # Tracking
    true_positive = 0
    samples = 0

    bar = tqdm(dataloader, leave=True)
    bar.set_description_str(f"Length: {length}")

    for inputs, targets, masks in bar:
        inputs, targets, masks = inputs.to('cuda'), targets.to('cuda'), masks.to('cuda')

        # Making prediction
        # with torch.cuda.amp.autocast():
        with torch.no_grad():
            outputs = model(inputs, masks)

            # Compute accuracy
            outputs_indices = torch.argmax(outputs, dim=-1) # -> [rotor, seq, batch]
            for rotor in range(outputs_indices.shape[0]):
                mask = outputs_indices[rotor][~masks.T] == targets[rotor][~masks.T]
                true_positive += mask.sum()
                samples += math.prod(mask.shape)

        # Set bar's postfix
        bar.set_postfix_str(f"Acc: {(true_positive / samples).item()}")

    # Output and record the result
    results[length] = (true_positive / samples).item()

Length: 25:  43%|████▎     | 15/35 [00:31<00:41,  2.06s/it, Acc: 0.9965651035308838]

Ploting the results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


x = list(results.keys())
y = list(results.values())

x, y = np.array(x), np.array(y)
print(x, y)

fig, ax = plt.subplots(figsize=(12, 10), dpi=80)
ax.bar(x,height=y)
plt.title('Seq5-45-large', fontsize=15)
ax.set_ylabel('Acc', fontsize=15)
ax.set_xlabel('Length', fontsize=15)
plt.show()
# ax.plot(bins, y, '--')

In [ ]:
plt.savefig('Seq5-45')